In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "e164b0b3-4881-4057-b875-5530eb164e35",
"fs.azure.account.oauth2.client.secret": '-m58Q~dmv99CQcH4Uqn0xnqHgRypVKiIfR8Zxb_9',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/b8bcf360-b8f3-419b-a4cd-2f7f7270843d/oauth2/token"}
base_path = "abfss://tokyo-olympic-kiran@tokyoolympickiran.dfs.core.windows.net/raw-data/"
transformed_data_path = "abfss://tokyo-olympic-kiran@tokyoolympickiran.dfs.core.windows.net/transformed-data/"

In [0]:
dbutils.fs.ls(base_path)

[FileInfo(path='abfss://tokyo-olympic-kiran@tokyoolympickiran.dfs.core.windows.net/raw-data/athletes.csv', name='athletes.csv', size=418492, modificationTime=1766742237000),
 FileInfo(path='abfss://tokyo-olympic-kiran@tokyoolympickiran.dfs.core.windows.net/raw-data/coaches.csv', name='coaches.csv', size=16889, modificationTime=1766742251000),
 FileInfo(path='abfss://tokyo-olympic-kiran@tokyoolympickiran.dfs.core.windows.net/raw-data/entriesGender.csv', name='entriesGender.csv', size=1123, modificationTime=1766742264000),
 FileInfo(path='abfss://tokyo-olympic-kiran@tokyoolympickiran.dfs.core.windows.net/raw-data/medals.csv', name='medals.csv', size=2414, modificationTime=1766742278000),
 FileInfo(path='abfss://tokyo-olympic-kiran@tokyoolympickiran.dfs.core.windows.net/raw-data/teams.csv', name='teams.csv', size=35270, modificationTime=1766742292000)]

In [0]:
spark

In [0]:
athletes_df = spark.read.format("csv").option("header","True").load(base_path+"athletes.csv")
coaches_df = spark.read.format("csv").option("header","True").load(base_path+"coaches.csv")
entriesGender_df = spark.read.format("csv").option("header","True").load(base_path+"entriesGender.csv")
medals_df = spark.read.format("csv").option("header","True").option("inferSchema","true").load(base_path+"medals.csv")
teams_df = spark.read.format("csv").option("header","True").load(base_path+"teams.csv")

In [0]:
athletes_df.show(10)

+-----------------+--------------------+-------------------+
|       PersonName|             Country|         Discipline|
+-----------------+--------------------+-------------------+
|  AALERUD Katrine|              Norway|       Cycling Road|
|      ABAD Nestor|               Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|               Italy|             Rowing|
|   ABALDE Alberto|               Spain|         Basketball|
|    ABALDE Tamara|               Spain|         Basketball|
|        ABALO Luc|              France|           Handball|
|     ABAROA Cesar|               Chile|             Rowing|
|    ABASS Abobakr|               Sudan|           Swimming|
| ABBASALI Hamideh|Islamic Republic ...|             Karate|
|    ABBASOV Islam|          Azerbaijan|          Wrestling|
+-----------------+--------------------+-------------------+
only showing top 10 rows


In [0]:
athletes_df.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches_df.show(10)

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
+--------------------+--------------------+-----------------+--------+
only s

In [0]:
coaches_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesGender_df.show(10)

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
|          Badminton|    86|  87|  173|
|  Baseball/Softball|    90| 144|  234|
|         Basketball|   144| 144|  288|
|   Beach Volleyball|    48|  48|   96|
|             Boxing|   102| 187|  289|
+-------------------+------+----+-----+
only showing top 10 rows


In [0]:
entriesGender_df.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
# Data transforation -->change type to interger
entriesGender_df = entriesGender_df.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Female").cast(IntegerType()))\
    .withColumn("Total",col("Female").cast(IntegerType()))

In [0]:
entriesGender_df.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals_df.show(10)

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
+----+--------------------+----+------+------+-----+-------------+
only showing top 10 rows


In [0]:
medals_df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
teams_df.show(10)

+-----------+--------------+--------------------+-----+
|   TeamName|    Discipline|             Country|Event|
+-----------+--------------+--------------------+-----+
|    Belgium|3x3 Basketball|             Belgium|  Men|
|      China|3x3 Basketball|People's Republic...|  Men|
|      China|3x3 Basketball|People's Republic...|Women|
|     France|3x3 Basketball|              France|Women|
|      Italy|3x3 Basketball|               Italy|Women|
|      Japan|3x3 Basketball|               Japan|  Men|
|      Japan|3x3 Basketball|               Japan|Women|
|     Latvia|3x3 Basketball|              Latvia|  Men|
|   Mongolia|3x3 Basketball|            Mongolia|Women|
|Netherlands|3x3 Basketball|         Netherlands|  Men|
+-----------+--------------+--------------------+-----+
only showing top 10 rows


In [0]:
teams_df.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals_df.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show(10)

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|               Italy|  10|
|             Germany|  10|
|              France|  10|
|         Netherlands|  10|
+--------------------+----+
only showing top 10 rows


In [0]:

# Calculate the average number of entries by gender for each discipline
from pyspark.sql.functions import when

average_entries_by_gender = entriesGender_df \
    .withColumn("Female", col("Female").cast("double")) \
    .withColumn("Male", col("Male").cast("double")) \
    .withColumn("Total", col("Total").cast("double")) \
    .withColumn(
        "Avg_Female",
        when(col("Total") != 0, col("Female") / col("Total")).otherwise(0)
    ) \
    .withColumn(
        "Avg_Male",
        when(col("Total") != 0, col("Male") / col("Total")).otherwise(0)
    )

average_entries_by_gender.show(10)

+-------------------+------+------+------+-------------------+------------------+
|         Discipline|Female|  Male| Total|         Avg_Female|          Avg_Male|
+-------------------+------+------+------+-------------------+------------------+
|     3x3 Basketball|  32.0|  32.0|  64.0|                0.5|               0.5|
|            Archery|  64.0|  64.0| 128.0|                0.5|               0.5|
|Artistic Gymnastics|  98.0|  98.0| 196.0|                0.5|               0.5|
|  Artistic Swimming| 105.0|   0.0| 105.0|                1.0|               0.0|
|          Athletics| 969.0|1072.0|2041.0| 0.4747672709456149|0.5252327290543851|
|          Badminton|  86.0|  87.0| 173.0|0.49710982658959535|0.5028901734104047|
|  Baseball/Softball|  90.0| 144.0| 234.0|0.38461538461538464|0.6153846153846154|
|         Basketball| 144.0| 144.0| 288.0|                0.5|               0.5|
|   Beach Volleyball|  48.0|  48.0|  96.0|                0.5|               0.5|
|             Bo

In [0]:
athletes_df.repartition(1).write.mode("overwrite").option("header","true").csv(transformed_data_path+"/athletes")

In [0]:
coaches_df.repartition(1).write.mode("overwrite").option("header","true").csv(transformed_data_path+"/coaches")
entriesGender_df.repartition(1).write.mode("overwrite").option("header","true").csv(transformed_data_path+"entriesgender")
medals_df.repartition(1).write.mode("overwrite").option("header","true").csv(transformed_data_path+"/medals")
teams_df.repartition(1).write.mode("overwrite").option("header","true").csv(transformed_data_path+"/teams")